# Experiments

### Setup

In [ ]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [1]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

Here is the RAG Application that we've been working with throughout this course

In [2]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

# TODO: Configure this model!
MODEL_NAME = "gpt-4o"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


USER_AGENT environment variable not set, consider setting it to identify your requests.


### Experiment

Here is a code snippet that should look similar to what you see from the starter code!

There are a few important components here.

1. We have defined an Evaluator
2. We pipe our dataset examples (dict) to the shape of input that our function `langsmith_rag` takes (str) using a target function

In [4]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "Dataset_1"

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-4o"
)

View the evaluation results for experiment: 'gpt-4o-3a076123' at:
https://smith.langchain.com/o/6b354ec6-3f46-4558-9cde-6a933975f725/datasets/1f9560b2-c8e9-4460-8d1e-9a99f230e018/compare?selectedSessions=c36abf0d-04b5-45d5-bd76-f1a0ea5068e8




c:\Users\bhavy\OneDrive\Desktop\College\3rd sem\MAT496\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
10it [01:02,  6.28s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,41.444713,0306a335-318c-49ae-8fc0-7c18f5fe916d,b8258152-82bb-4296-9254-df4151a676e1
1,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,2.074160,2f2966e9-11b2-44c5-9afe-4fad843c0c20,ade1856d-dd82-4d51-9428-a6838c818a2f
2,How do I pass metadata in with @traceable?,To pass metadata with the `@traceable` decorat...,None,You can pass metadata with the @traceable deco...,1,2.453570,526dcd70-a817-4246-bdc9-9d63ff2e1cda,3eba5668-cb4c-4252-9057-ba16a2995a08
3,How do I set up tracing to LangSmith if I'm us...,"To set up tracing to LangSmith with LangChain,...",None,To set up tracing to LangSmith while using Lan...,1,3.369347,78efc721-8384-4175-8dbd-da3adf38385f,3e27b1cb-0390-4581-8894-dc1aa30d15e9
4,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,2.879833,7e152e37-d966-45de-9dd6-3ff554fe2693,d8fef360-fcf9-4ca3-a31d-fd58d6fe16f7
5,Can LangSmith be used for finetuning and model...,"No, LangSmith is primarily a platform for obse...",None,"Yes, LangSmith can be used for fine-tuning and...",1,1.887516,8e1fddd0-eabb-42d5-8d17-0ff3dd9aced2,0dcb6c65-fddb-4f74-aa11-79003b62b5d7
6,What is LangSmith used for in three sentences?,LangSmith is used for building production-grad...,None,LangSmith is a platform designed for the devel...,1,2.572325,a5d2efac-09b1-4faf-bf21-4162aa00c44f,748bbaf5-02a4-48c4-86b3-2f9a9e99f327
7,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,2.096829,d5f61dcb-2657-4dac-bad0-475ee89d851d,1c583067-9f44-4a17-9b6e-0159d9fe64dd
8,What testing capabilities does LangSmith have?,LangSmith allows users to run multiple experim...,None,LangSmith offers capabilities for creating dat...,1,1.884860,e982f99f-81af-49e0-8f60-7b7a546d906d,dfb00758-9b5f-4e21-a13b-4c71af34ef90
9,Does LangSmith support offline evaluation?,"No, LangSmith focuses on online evaluations, p...",None,"Yes, LangSmith supports offline evaluation thr...",1,1.575760,f57fa9bf-bf79-471a-af5a-a07172ea916d,05bfbd1b-86ff-40f8-a307-ac4c20693e9c


### Modifying your Application

Now, let's change our model to gpt-35-turbo and see how it performs!

Make this change, and then run this code snippet!

In [5]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

View the evaluation results for experiment: 'gpt-3.5-turbo-7440f1c4' at:
https://smith.langchain.com/o/6b354ec6-3f46-4558-9cde-6a933975f725/datasets/1f9560b2-c8e9-4460-8d1e-9a99f230e018/compare?selectedSessions=c99b41b5-271c-4e84-b081-99484969163c




2it [00:04,  2.24s/it]Error running target function: Error code: 400 - {'error': {'message': 'something went wrong reading your request', 'type': 'invalid_request_error', 'param': None, 'code': None}}
Traceback (most recent call last):
  File "c:\Users\bhavy\OneDrive\Desktop\College\3rd sem\MAT496\venv\Lib\site-packages\langsmith\evaluation\_runner.py", line 1924, in _forward
    fn(*args, langsmith_extra=langsmith_extra)
  File "c:\Users\bhavy\OneDrive\Desktop\College\3rd sem\MAT496\venv\Lib\site-packages\langsmith\run_helpers.py", line 703, in wrapper
    function_result = run_container["context"].run(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\bhavy\AppData\Local\Temp\ipykernel_23256\862480817.py", line 5, in target_function
    return langsmith_rag(inputs["question"])
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\bhavy\OneDrive\Desktop\College\3rd sem\MAT496\venv\Lib\site-packages\langsmith\run_helpers.py", line 703, in wrapper
    function

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id,feedback.wrapper
0,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1.0,2.492683,0306a335-318c-49ae-8fc0-7c18f5fe916d,b3c600d4-658d-4adb-8153-48e8e062e73e,NaN
1,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1.0,2.052750,2f2966e9-11b2-44c5-9afe-4fad843c0c20,9d6b9732-ca50-4b51-a1e3-e29782dead72,NaN
2,How do I pass metadata in with @traceable?,None,"BadRequestError(""Error code: 400 - {'error': {...",You can pass metadata with the @traceable deco...,NaN,115.299205,526dcd70-a817-4246-bdc9-9d63ff2e1cda,6e1c1c7a-7faa-4979-9c36-2f9699c3427a,NaN
3,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith using LangChain...,None,To set up tracing to LangSmith while using Lan...,0.0,3.145344,78efc721-8384-4175-8dbd-da3adf38385f,96d73696-1797-4aa1-9835-1a9f56e44c44,NaN
4,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1.0,3.041513,7e152e37-d966-45de-9dd6-3ff554fe2693,77ed38ce-5649-449e-98d2-56ffa7305c66,NaN
5,Can LangSmith be used for finetuning and model...,LangSmith is designed for LLM observability an...,None,"Yes, LangSmith can be used for fine-tuning and...",1.0,2.138430,8e1fddd0-eabb-42d5-8d17-0ff3dd9aced2,288cd61f-9067-43fe-adeb-d45389040a2e,NaN
6,What is LangSmith used for in three sentences?,LangSmith is a platform for building productio...,None,LangSmith is a platform designed for the devel...,1.0,2.871908,a5d2efac-09b1-4faf-bf21-4162aa00c44f,1194123b-413d-4709-a42e-038518298f0b,NaN
7,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1.0,119.187324,d5f61dcb-2657-4dac-bad0-475ee89d851d,e4c38734-05d0-4f4b-8a26-23e0b7f96297,NaN
8,What testing capabilities does LangSmith have?,LangSmith allows you to run multiple experimen...,None,LangSmith offers capabilities for creating dat...,1.0,1.874613,e982f99f-81af-49e0-8f60-7b7a546d906d,7c523988-72df-4db6-9b1c-6d35cfbd2c65,NaN
9,Does LangSmith support offline evaluation?,"No, LangSmith primarily supports online evalua...",None,"Yes, LangSmith supports offline evaluation thr...",1.0,1.494201,f57fa9bf-bf79-471a-af5a-a07172ea916d,309ca5e2-49d3-462e-8460-c1528fa28a25,NaN


In [6]:
from langsmith import traceable, evaluate, Client
from openai import OpenAI
from typing import List

# Configuration
MODEL_NAME = "gpt-4o"
MODEL_PROVIDER = "openai"
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

client = Client()
openai_client = OpenAI()

# Core RAG functions
@traceable(run_type="chain")
def retrieve_documents(question: str):
    # Your retrieval logic here
    return []  # Mock documents

@traceable(
    run_type="llm",
    metadata={"ls_provider": MODEL_PROVIDER, "ls_model_name": MODEL_NAME}
)
def call_openai(messages: List[dict]):
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {"role": "system", "content": RAG_SYSTEM_PROMPT},
        {"role": "user", "content": f"Context: {formatted_docs}\n\nQuestion: {question}"}
    ]
    return call_openai(messages)

@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content

### Running over Different pieces of Data

##### Dataset Version

You can execute an experiment on a specific version of a dataset in the sdk by using the `as_of` parameter in `list_examples`

Let's try running on just our initial dataset.

In [7]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, as_of="initial dataset"),   # We use as_of to specify a version
    evaluators=[is_concise_enough],
    experiment_prefix="initial dataset version"
)

StopIteration: 

##### Dataset Split

You can run an experiment on a specific split of your dataset, let's try running on the Crucial Examples split.

In [8]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, splits=["Crucial Examples"]),  # We pass in a list of Splits
    evaluators=[is_concise_enough],
    experiment_prefix="Crucial Examples split"
)

StopIteration: 

##### Specific Data Points

You can specify individual data points to run an experiment over as well

In [9]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[   # We pass in a specific list of example_ids
            # TODO: You will need to paste in your own example ids for this to work!
            "",
            ""
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

LangSmithError: Failed to GET /examples in LangSmith API. HTTPError('422 Client Error: unknown for url: https://api.smith.langchain.com/examples?offset=0&id=&id=&inline_s3_urls=True&limit=100&dataset=1f9560b2-c8e9-4460-8d1e-9a99f230e018', '{"detail":["query.id.0: Input should be a valid UUID, invalid length: expected length 32 for simple format, found 0","query.id.1: Input should be a valid UUID, invalid length: expected length 32 for simple format, found 0"]}')

### Other Parameters

##### Repetitions

You can run an experiment several times to make sure you have consistent results

In [10]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="two repetitions",
    num_repetitions=2   # This field defaults to 1
)

View the evaluation results for experiment: 'two repetitions-ec5afbb0' at:
https://smith.langchain.com/o/6b354ec6-3f46-4558-9cde-6a933975f725/datasets/1f9560b2-c8e9-4460-8d1e-9a99f230e018/compare?selectedSessions=f7ccec71-1267-499f-ace7-9198bfbec017




20it [00:34,  1.73s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation.",None,"Yes, LangSmith supports online evaluation as a...",1,1.056642,0306a335-318c-49ae-8fc0-7c18f5fe916d,59527408-03ed-4554-bdb3-ce0f261cc134
1,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.144726,2f2966e9-11b2-44c5-9afe-4fad843c0c20,1f36ebd4-ea6d-4547-be14-eed34a083d5d
2,How do I pass metadata in with @traceable?,"To pass metadata with `@traceable`, you can us...",None,You can pass metadata with the @traceable deco...,1,2.459387,526dcd70-a817-4246-bdc9-9d63ff2e1cda,1d7c6096-21df-4c3c-910f-f359ecb2d44d
3,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,1,3.915445,78efc721-8384-4175-8dbd-da3adf38385f,a75d3994-b575-4554-bb3a-9a5723ea2d4d
4,How do I create user feedback with the LangSmi...,I don't know.,None,To create user feedback with the LangSmith SDK...,1,0.797915,7e152e37-d966-45de-9dd6-3ff554fe2693,efda32fb-cd7c-4409-b095-a3c83b78bb12
5,Can LangSmith be used for finetuning and model...,"No, LangSmith is not designed for finetuning o...",None,"Yes, LangSmith can be used for fine-tuning and...",1,1.278775,8e1fddd0-eabb-42d5-8d17-0ff3dd9aced2,acdf9dd2-04e5-4bf6-a2aa-1ee7970dc1a9
6,What is LangSmith used for in three sentences?,LangSmith is used to streamline the developmen...,None,LangSmith is a platform designed for the devel...,1,2.476902,a5d2efac-09b1-4faf-bf21-4162aa00c44f,23a60914-8619-443e-ba50-c729d74d3676
7,How can I trace with the @traceable decorator?,The `@traceable` decorator is used to trace or...,None,To trace with the @traceable decorator in Pyth...,1,1.842155,d5f61dcb-2657-4dac-bad0-475ee89d851d,805fba69-8f0e-4fbc-97ae-4d28456338ae
8,What testing capabilities does LangSmith have?,LangSmith has testing capabilities that allow ...,None,LangSmith offers capabilities for creating dat...,1,1.463349,e982f99f-81af-49e0-8f60-7b7a546d906d,92f5767f-1e19-40bb-b5db-1d2c940721b7
9,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation.",None,"Yes, LangSmith supports offline evaluation thr...",1,0.843642,f57fa9bf-bf79-471a-af5a-a07172ea916d,a9841019-1629-4fb2-b62a-cff911efba99


##### Concurrency
You can also kick off concurrent threads of execution to make your experiments finish faster!

In [11]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="concurrency",
    max_concurrency=3,  # This defaults to None, so this is an improvement!
)

View the evaluation results for experiment: 'concurrency-5421f623' at:
https://smith.langchain.com/o/6b354ec6-3f46-4558-9cde-6a933975f725/datasets/1f9560b2-c8e9-4460-8d1e-9a99f230e018/compare?selectedSessions=73f6ccc3-1fde-4d1e-b4bd-2561ae086502




10it [00:06,  1.51it/s]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation.",None,"Yes, LangSmith supports online evaluation as a...",1,0.782395,0306a335-318c-49ae-8fc0-7c18f5fe916d,6ae351c0-2b76-46f7-963e-343e0a001c24
1,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.458477,2f2966e9-11b2-44c5-9afe-4fad843c0c20,0f7ebf33-9d65-4192-9cfc-ffa7ed4e05d4
2,How do I pass metadata in with @traceable?,You can pass metadata in with `@traceable` by ...,None,You can pass metadata with the @traceable deco...,1,1.932800,526dcd70-a817-4246-bdc9-9d63ff2e1cda,539a8199-6711-4af0-b82b-e7ea257d267c
3,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith if you're using...,None,To set up tracing to LangSmith while using Lan...,1,1.889353,78efc721-8384-4175-8dbd-da3adf38385f,019e09b3-9459-493d-89f7-3525f270b761
4,Can LangSmith be used for finetuning and model...,"No, LangSmith is not intended for finetuning o...",None,"Yes, LangSmith can be used for fine-tuning and...",1,1.172049,8e1fddd0-eabb-42d5-8d17-0ff3dd9aced2,0ef3f178-e20d-46f5-935a-f529cf717095
5,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.508440,7e152e37-d966-45de-9dd6-3ff554fe2693,f87ccb27-dccb-4f87-af0e-828ebd1e8d21
6,What is LangSmith used for in three sentences?,"LangSmith is used for developing, evaluating, ...",None,LangSmith is a platform designed for the devel...,1,2.036944,a5d2efac-09b1-4faf-bf21-4162aa00c44f,ffbe3916-c9e1-4af6-a826-41df2cc28b20
7,Does LangSmith support offline evaluation?,"No, LangSmith does not support offline evaluat...",None,"Yes, LangSmith supports offline evaluation thr...",1,0.778809,f57fa9bf-bf79-471a-af5a-a07172ea916d,02ddf033-059c-47da-a996-cb5001f4a971
8,How can I trace with the @traceable decorator?,"To trace with the `@traceable` decorator, you ...",None,To trace with the @traceable decorator in Pyth...,1,2.718285,d5f61dcb-2657-4dac-bad0-475ee89d851d,f97a87f9-127c-466e-a7c4-7f0c1f4f92a3
9,What testing capabilities does LangSmith have?,LangSmith offers testing capabilities for anal...,None,LangSmith offers capabilities for creating dat...,1,2.179770,e982f99f-81af-49e0-8f60-7b7a546d906d,2c9fd429-bfb2-41a5-8ab0-f3c042610e60


##### Metadata 

You can (and should) add metadata to your experiments, to make them easier to find in the UI

In [12]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="metadata added",
    metadata={  # We can pass custom metadata for the experiment, such as the model name
        "model_name": MODEL_NAME
    }
)

View the evaluation results for experiment: 'metadata added-c17164ca' at:
https://smith.langchain.com/o/6b354ec6-3f46-4558-9cde-6a933975f725/datasets/1f9560b2-c8e9-4460-8d1e-9a99f230e018/compare?selectedSessions=011ce4ba-d79e-4862-8827-effdf0a11547




10it [00:17,  1.72s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation.",None,"Yes, LangSmith supports online evaluation as a...",1,0.699183,0306a335-318c-49ae-8fc0-7c18f5fe916d,83924472-be4b-475a-b944-480c8913ef30
1,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.293176,2f2966e9-11b2-44c5-9afe-4fad843c0c20,5cb49419-9e2b-4924-8030-3203f91b99cb
2,How do I pass metadata in with @traceable?,To pass metadata with the `@traceable` annotat...,None,You can pass metadata with the @traceable deco...,1,1.582009,526dcd70-a817-4246-bdc9-9d63ff2e1cda,f23ab45f-5aa5-4bb3-b8ff-b1c1e116e0c8
3,How do I set up tracing to LangSmith if I'm us...,To set up tracing with LangSmith while using L...,None,To set up tracing to LangSmith while using Lan...,1,2.673400,78efc721-8384-4175-8dbd-da3adf38385f,4cea0bef-8efb-4cb4-894e-b4e47f12b6c6
4,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.950975,7e152e37-d966-45de-9dd6-3ff554fe2693,4e3f418d-7c06-4c12-9b05-68d079d56643
5,Can LangSmith be used for finetuning and model...,"No, LangSmith is not used for finetuning and m...",None,"Yes, LangSmith can be used for fine-tuning and...",1,1.243838,8e1fddd0-eabb-42d5-8d17-0ff3dd9aced2,10cd84e0-1d59-4df0-8987-a06f6dbec5fc
6,What is LangSmith used for in three sentences?,"LangSmith is used for developing, testing, deb...",None,LangSmith is a platform designed for the devel...,1,2.919292,a5d2efac-09b1-4faf-bf21-4162aa00c44f,272c4ebc-7f37-4370-b855-1d34c05f5c6f
7,How can I trace with the @traceable decorator?,The `@traceable` decorator is used to enable t...,None,To trace with the @traceable decorator in Pyth...,1,1.657677,d5f61dcb-2657-4dac-bad0-475ee89d851d,d45befe4-5cb0-4c19-a371-2ffe80e534f6
8,What testing capabilities does LangSmith have?,LangSmith offers testing capabilities that inc...,None,LangSmith offers capabilities for creating dat...,1,1.574833,e982f99f-81af-49e0-8f60-7b7a546d906d,f957484d-a544-4819-a9cf-f93dc5b81be5
9,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation.",None,"Yes, LangSmith supports offline evaluation thr...",1,0.742602,f57fa9bf-bf79-471a-af5a-a07172ea916d,a327dfd2-bb05-425a-899a-40fdbb6e535f
